<a href="https://colab.research.google.com/github/andrePankraz/speech_service/blob/main/notebooks/NLLB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Translation - No Language Left Behind (NLLB)
The following Notebook can translate text between 200 languages. It's based on the Meta model [NLLB](https://ai.facebook.com/research/no-language-left-behind/).

# Set-up environment
We need following packages:

*   optional: accelerate and bitsandbytes for [8bit model inference](https://huggingface.co/blog/hf-bitsandbytes-integration)
*   [transformers](https://github.com/huggingface/transformers) for NLLB model
*   [sentence_cleaner_splitter](https://github.com/facebookresearch/LASER/tree/main/utils) from project [LASER](https://github.com/facebookresearch/LASER) for sentence splitting

In [1]:
# !pip install --quiet accelerate
# !pip install --quiet bitsandbytes
!pip install --quiet git+https://github.com/huggingface/transformers.git # Install latest version of transformers
!pip install --quiet sentence_cleaner_splitter@git+https://github.com/facebookresearch/LASER.git#subdirectory=utils

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


# Check GPU
The following experiments can be run without a GPU, but it will take much longer!

See Colab Menu: Runtime / Change type.

Clean up dangling pointers and VRAM and check if GPU available:


In [64]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

!nvidia-smi

Wed Nov  9 19:20:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    28W /  70W |   3182MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Set-up AI-model for translation
We need a pre-trained tokenizer and a model, that are downloaded from [Hugging Faces](https://huggingface.co/models?sort=downloads&search=facebook%2Fnllb). 

Multiple model sizes are available, to adapt to the GPU VRAM constraints. Only the big models provide enough quality, the smaller are just for quick tests.

With the param `device_map='auto'` instead of `.cuda()` you can dynamically spread the model over GPUs / CPUs and load bigger models, but they are slower as inference time. See [accelerate](https://huggingface.co/docs/accelerate/usage_guides/big_modeling).

With the param `load_in_8bit=True` you can also load bigger models, but they are slow at inference time and I also had inference bugs with NLLB. See [bitsandbytes](https://huggingface.co/blog/hf-bitsandbytes-integration).
(This also doesn't always work: 'ERROR: Your GPU does not support Int8 Matmul!')

In [107]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

# Load model NLLB
# facebook/nllb-200-distilled-600M, facebook/nllb-200-distilled-1.3B, facebook/nllb-200-3.3B
# VRAM at least: 4 | 8 | 16 GB VRAM
model_id = 'facebook/nllb-200-distilled-600M'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).cuda()

model.device

Some weights of M2M100ForConditionalGeneration were not initialized from the model checkpoint at facebook/nllb-200-distilled-600M and are newly initialized: ['model.decoder.embed_positions.weights', 'model.encoder.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


device(type='cuda', index=0)

A source and a target language are to be provided to the translation pipeline. The supported language IDs are visible with this map from project [LASER](https://github.com/facebookresearch/LASER), which has been used for NLLB training.

In [108]:
# Optional output...
from sentence_cleaner_splitter.sentence_split import split_lang_code_map

split_lang_code_map

{'ace_Arab': 'ace_Arab',
 'ace_Latn': 'ace_Latn',
 'acm_Arab': 'acm',
 'acq_Arab': 'acq',
 'aeb_Arab': 'aeb',
 'afr_Latn': 'afr',
 'ajp_Arab': 'ajp',
 'aka_Latn': 'aka',
 'amh_Ethi': 'amh',
 'apc_Arab': 'apc',
 'arb_Arab': 'ara_Arab',
 'arb_Latn': 'ara_Latn',
 'ars_Arab': 'ars',
 'ary_Arab': 'ary',
 'arz_Arab': 'arz',
 'asm_Beng': 'asm',
 'ast_Latn': 'ast',
 'awa_Deva': 'awa',
 'ayr_Latn': 'ayr',
 'azb_Arab': 'azb',
 'azj_Latn': 'azj',
 'bak_Cyrl': 'bak',
 'bam_Latn': 'bam',
 'ban_Latn': 'ban',
 'bel_Cyrl': 'bel',
 'bem_Latn': 'bem',
 'ben_Beng': 'ben',
 'bho_Deva': 'bho',
 'bjn_Arab': 'bjn_Arab',
 'bjn_Latn': 'bjn_Latn',
 'bod_Tibt': 'bod',
 'bos_Latn': 'bos',
 'bug_Latn': 'bug',
 'bul_Cyrl': 'bul',
 'cat_Latn': 'cat',
 'ceb_Latn': 'ceb',
 'ces_Latn': 'ces',
 'cjk_Latn': 'cjk',
 'ckb_Arab': 'ckb',
 'crh_Latn': 'crh_Latn',
 'cym_Latn': 'cym',
 'dan_Latn': 'dan',
 'deu_Latn': 'deu',
 'dik_Latn': 'dik',
 'diq_Latn': 'diq',
 'dyu_Latn': 'dyu',
 'dzo_Tibt': 'dzo',
 'ell_Grek': 'ell',
 'eng

In [109]:
src_lang='deu_Latn'
tgt_lang='rus_Cyrl'

src_text = '''
Menschheitstraum
Das Verstehen einer Sprache, ohne sie gelernt zu haben, ist ein alter Menschheitstraum (Turmbau zu Babel, J. Bechers numerische Interlingua, Timerio, Babelfisch, Pfingstwunder, Science-Fiction-Geschichten). Die Erfindung der Computer in Kombination mit der Beschäftigung mit dem Phänomen Sprache als wissenschaftliche Disziplin (Sprachwissenschaft) hat zum ersten Mal einen konkreten Weg zur Erfüllung dieses Traums geöffnet.

Geschichte
Bis zum heutigen Tag hat das militärische Interesse den Weg der MÜ entscheidend geprägt. Eines der frühesten Projekte war ein Russisch-Englisch-Übersetzungsprogramm für das US-Militär. Trotz seiner anekdotenhaft schlechten Qualität genoss das Programm hohe Popularität unter US-Militärs, die sich zum ersten Mal ohne den Umweg über Dritte (Dolmetscher und Übersetzer) selbst zumindest einen Eindruck vom Inhalt russischer Dokumente verschaffen konnten.

Der 1966 für das Verteidigungsministerium der Vereinigten Staaten erstellte ALPAC-Bericht[1] bescheinigte der MÜ grundsätzliche Unrealisierbarkeit und brachte mit einem Schlag die Forschung für fast 20 Jahre praktisch ganz zum Erliegen. Erst in den 1980er Jahren begannen Elektrokonzerne wie die Siemens AG (Metal-Projekt) erneut mit der Forschung. Zu diesen Vorhaben zählt auch die Forschungsarbeit im Sonderforschungsbereich „Elektronische Sprachforschung“ an der Universität des Saarlandes. Hier wurde das System „SUSY“ entwickelt, das in der Lage war, aus dem Deutschen und ins Deutsche zu übersetzen.[2] Ein weiteres System des Sonderforschungsbereichs war ASCOF, in dem neben morpho-syntaktischen auch semantische Informationen für die Übersetzung herangezogen wurden.[3] In der gleichen Zeit initiierte die japanische Regierung das Fünfte-Generation-Projekt, bei dem MÜ vom Englischen ins Japanische zunächst auf der Basis der Programmiersprache Prolog implementiert wurde. Die enge Zusammenarbeit zwischen Universitäten, Elektrokonzernen und Regierung führte zu den weltweit ersten kommerziellen MÜ-Programmen für PCs und hat Japan in die Führungsposition der MÜ-Forschung weltweit gebracht. In den 1990er Jahren lief in Deutschland das BMBF-Leitprojekt Verbmobil, dessen Ziel es war, deutsche, englische und japanische gesprochene Dialogsprache zu dolmetschen. Das Verbmobil-System sollte gesprochene Spontansprache erkennen, die Eingabe analysieren, übersetzen, einen Satz erzeugen und ihn aussprechen.[4]

In den 2000er Jahren kamen vermehrt statistische Verfahren zum Einsatz. So bietet Google seit 2006 ein statistisches Übersetzungssystem an.[5] Auch regelbasierte Ansätze wurden weiterentwickelt. Eines der bekanntesten Forschungsprojekte dieser Art ist die freie Software Apertium, die von der spanischen Regierung und der Regierung von Katalonien finanziert und an der Universität Alicante weiterentwickelt wird.

Der Stand der MÜ im Jahr 2010 wurde von vielen Menschen als unbefriedigend bewertet. Grundsätzlich versteht die Wissenschaft menschliche Sprache aber noch unzureichend. Die meisten Sprachwissenschaftler gingen gar davon aus, dass maschineller Übersetzung ohne über das reine Sprachverständnis weit hinausgehende Kompetenzen automatischer Systeme grundsätzliche Grenzen gesetzt sind, da viele Übersetzungen zudem große Mengen an konzeptuellem Wissen, Metawissen sowie Kenntnisse über die Konstitution menschlicher Umwelt allgemein und über die Konventionen sozialer Interaktion erfordern.

Seit dem Jahr 2016 werden für Übersetzungsprogramme zunehmend künstliche neuronale Netze, d. h. künstliche Intelligenzen eingesetzt, wodurch der Fortschritt rasant zunahm. Beispiele sind DeepL, Google Übersetzer, Yandex.Translate sowie der Bing Translator, die fortan deutlich bessere Ergebnisse erzielten.[6]

Im März 2018 teilte Microsoft mit, durch eine KI Chinesisch-Englisch-Übersetzungen mit der Qualität eines professionellen menschlichen Übersetzers zu erreichen. Das sei ein Durchbruch bei der maschinellen Übersetzung, den Microsoft nicht so früh erwartet habe.[7][8]

Der Bedarf an MÜ-Anwendungen steigt weiter:

Viele Texte sind heute digital verfügbar (also leicht für den Computer zu verarbeiten).
Die Globalisierung erfordert die Übertragung von immer mehr Texten in immer mehr Sprachen (der Markt für Übersetzung verdoppelt sich alle vier Jahre), während die Popularität des Berufs des Übersetzers/Dolmetschers stagniert.
Gerade von nur wenigen Westeuropäern/Amerikanern gesprochene beziehungsweise für diese schwierig zu erlernende Sprachen aus Regionen, deren Bewohner ihrerseits kaum westliche Sprachen sprechen, werden immer wichtiger:
kommerziell wichtig: die ostasiatischen Sprachen Chinesisch, Koreanisch und Japanisch; sowie Thai.
militärisch wichtig: Sprachen der internationalen Konfliktregionen, vor allem mit Beteiligung des US-Militärs. 2003 haben gleich mehrere US-Software-Unternehmen Übersetzungsprogramme für Arabisch und Paschtu (eine der Sprachen in Afghanistan und Grenzregionen Pakistans) herausgebracht. Ebenfalls 2003 hat die DARPA einen Blind-Wettbewerb für eine unbekannte Ausgangssprache durchgeführt. 2011 wurde das BOLT-Programm gestartet, das zum Ziel hat, die Erforschung der Übersetzung chinesischer und arabischer Texte ins Englische zu fördern.[9][10]
'''

# Set-up translation pipeline

In [110]:
translation_pipeline = pipeline('translation',
                                model=model,
                                tokenizer=tokenizer,
                                device=model.device,
                                src_lang=src_lang,
                                tgt_lang=tgt_lang,
                                max_length=512,
                                batch_size=16,
                                # beam search with early stopping, better than greedy:
                                num_beams=3,
                                early_stopping=True)

# First Translation Test
Very long sequences will run into the following problems:
*    'Token indices sequence length is longer than the specified maximum sequence length for this model (1320 > 1024).'
*    'Your input_length: 1320 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)'

The model is trained on sentence level and problems are possible with longer sequences. So it's a good idea to split the text.

See paper: 'The maximum sequence length during training is 512 for both the encoder and the decoder.'

The model is restricted to 512 tokens, which means less than 500 words! Text documents should be split into sentences.

In [112]:
translation_pipeline(src_text)

Your input_length: 1320 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


[{'translation_text': 'Человеческая мечта Понимание языка, не изучая его, также открыло для себя впервые конкретный путь к достижению этой мечты. История современного общества требует все большего военного интереса к китайскому языку (цифровая интерлингва, Тимерио, Бабелфиш, Пингвин-Транс, история научной фантастики). Создание компьютеров в сочетании с использованием феномена языка в качестве научной дисциплины (языковая наука) впервые открыло для себя конкретный путь к достижению этой мечты. История современного общества требует все большего военного интереса к китайскому языку. Один из первых проектов, созданных для изучения английского языка для американских жителей, был разработан в течение многих лет. Несмотря на практические результаты исследований в области английского языка, а также исследования в области изучения английского языка, проект "Бингвин-Транс-Транс-Транс-Транс-Транс-Транс-Транс-Транс-Пренс-Транс-Транс-Транс-Транс-Транс-Пренс-Транс-Транс-Пренс-Транс-Транс-Транс-Пренс

# Set-up Sentence Splitter
Use sentence splitter form project [LASER](https://github.com/facebookresearch/LASER), which has been used for NLLB training. This is a small AI-model, which will be downloaded in the background. Splitting sentences for 200 languages is much harder than a Regexp...

In [113]:
from sentence_cleaner_splitter.cleaner_splitter import SentenceSplitClean

sentence_splitter = SentenceSplitClean(src_lang, 'default')

# Split text into sentences
Replace or remove some special chars like zero-width spaces or non-breaking spaces. The NLLB model doesn't understand them.

In [114]:
norm_texts = []
for _, _, line in sentence_splitter(src_text.replace('\u200b', ' ')):
    norm_texts.append(line.strip())

# Translate Sentences
Attention: The NLLB model doesn't always map empty lines to empty lines in other languages! This seems to be a bug in the model? We have to postprocess this or insert a different input token for empty lines.

In [116]:
tgt_texts = translation_pipeline(norm_texts)
tgt_texts = [t['translation_text'] for t in tgt_texts]

tgt_texts

['Мечта человечества',
 'Понимание языка, не изучая его, - это древняя мечта человечества (Строительство Вавилонской башни, Цифровая интерлингва Дж. Бекера, Тимерьо, Вавилонская рыба, Пингвинские чудеса, научные фантастики).',
 'Изобретение компьютеров в сочетании с изучением языка как научной дисциплины (лингвистика) впервые открыло конкретный путь к осуществлению этой мечты.',
 'С другой стороны:',
 'История',
 'На сегодняшний день военные интересы определяют путь МУ.',
 'Одним из самых ранних проектов была программа перевода на русско-английский язык для американских военных.',
 'Несмотря на плохое анекдотное качество, программа была очень популярна среди военнослужащих США, которые впервые смогли получить, по крайней мере, собственное представление о содержании российских документов без использования третьих лиц (переводчиков и переводчиков).',
 'С другой стороны:',
 'Отчет ALPAC, подготовленный в 1966 году для Министерства обороны Соединенных Штатов[1], подтвердил, что МУ в принци

# Without Pipeline
As alternative to pipelines, we can also use lower level functions for inference.

(With larger batch and beam sizes we get into VRAM problems really fast with the current HuggingFaces implementation!)

In [117]:
tokenizer.src_lang = src_lang
batch_size = 8
tgt_texts = []
with torch.inference_mode():
  for i in range(0, len(norm_texts), batch_size):
    encoding = tokenizer(
      norm_texts[i:i+batch_size],
      truncation=True,
      padding=True,
      return_tensors="pt").to(model.device)

    translated_tokens = model.generate(
        **encoding,
        forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang],
        max_length=512,
        # beam search with early stopping, better than greedy:
        num_beams=3, 
        early_stopping=True)
    tgt_texts += tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

tgt_texts

['Мечта человечества',
 'Понимание языка, не изучая его, - это древняя мечта человечества (Строительство Вавилонской башни, Цифровая интерлингва Дж. Бекера, Тимерьо, Вавилонская рыба, Пингвинские чудеса, научные фантастики).',
 'Изобретение компьютеров в сочетании с изучением языка как научной дисциплины (лингвистика) впервые открыло конкретный путь к осуществлению этой мечты.',
 'С другой стороны:',
 'История',
 'На сегодняшний день военные интересы определяют путь МУ.',
 'Одним из самых ранних проектов была программа перевода на русско-английский язык для американских военных.',
 'Несмотря на плохое анекдотное качество, программа была очень популярна среди военнослужащих США, которые впервые смогли получить, по крайней мере, собственное представление о содержании российских документов без использования третьих лиц (переводчиков и переводчиков).',
 'С другой стороны:',
 'Отчет ALPAC, подготовленный в 1966 году для Министерства обороны Соединенных Штатов[1], подтвердил, что МУ в принци

# Show sentences and their matching translation

In [118]:
for s, t in zip(norm_texts, tgt_texts):
  if len(s):
    print(f"{s!r} ---> {t!r}")

'Menschheitstraum' ---> 'Мечта человечества'
'Das Verstehen einer Sprache, ohne sie gelernt zu haben, ist ein alter Menschheitstraum (Turmbau zu Babel, J. Bechers numerische Interlingua, Timerio, Babelfisch, Pfingstwunder, Science-Fiction-Geschichten).' ---> 'Понимание языка, не изучая его, - это древняя мечта человечества (Строительство Вавилонской башни, Цифровая интерлингва Дж. Бекера, Тимерьо, Вавилонская рыба, Пингвинские чудеса, научные фантастики).'
'Die Erfindung der Computer in Kombination mit der Beschäftigung mit dem Phänomen Sprache als wissenschaftliche Disziplin (Sprachwissenschaft) hat zum ersten Mal einen konkreten Weg zur Erfüllung dieses Traums geöffnet.' ---> 'Изобретение компьютеров в сочетании с изучением языка как научной дисциплины (лингвистика) впервые открыло конкретный путь к осуществлению этой мечты.'
'Geschichte' ---> 'История'
'Bis zum heutigen Tag hat das militärische Interesse den Weg der MÜ entscheidend geprägt.' ---> 'На сегодняшний день военные интерес